# Application of Gibbs Sampling to Mixture of Gaussians

## Task
The file `data_ex06.csv` contains data sampled from a mixture of $K = 4$ Gaussians. The standard deviations of the Gaussians are known and
identical, $\sigma_k = \sigma = 0.5$. Reconstruct the unknown parameters and the latent variables:
$$
    p_k, \mu_k, Z_i
$$
where $Z_i$ represents the assignment of each point to one of the mixture components. To perform the inference, use _Gibbs sampling_.

**Step 1** <br>
Initialize randomly $p_k, \mu_k, Z_i$. Compute $N_k = \sum_i \chi(Z_i=k)$ and $m_k=\sum_i \chi(Z_i=k) x_k$.

**Step 2** <br>
Perform Gibbs sampling iterations:
- Sample $\mu_k$ from a normal with mean:
  $$
    \mu'_k = \left(\frac{\mu_0}{\sigma_0}+\frac{m_k}{\sigma}\right)\frac{1}{\left(\frac{1}{\sigma_0^2}+\frac{N_k}{\sigma^2}\right)}
  $$
  and standard deviation:
  $$
    \sigma'_k = \left(\frac{1}{\sigma_0^2}+\frac{N_k}{\sigma^2}\right)^{-\frac{1}{2}}
  $$
  Here $\mu_0 = 0$ and $\sigma_0 = 1000$ are the Gaussian prior parameters. Do this for all $k$.

- Sample the $p$ from a Dirichlet distribution with parameters $\gamma'_k=\gamma_k+N_k$. Here $\gamma_k=1$ are the Dirichlet prior parameters.

- Sample the $Z_i$ from a categorical distribution:
  $$
    P(Z_i=k) = \frac{p_k\exp\left\{-\frac{(x_k-\mu_k)^2}{2\sigma^2}\right\}}{\sum_k p_k\exp\left\{-\frac{(x_k-\mu_k)^2}{2\sigma^2}\right\}}
  $$

- If $Z_i$ is updated, update $N_k$ and $m_k$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt